#### 0. Import basic methods (a.o. from source2.py)

In [1]:
from source2 import *
import numpy as np

In [2]:
# load basic info from the Input sheet
mapping = load_input_from_xls(filename=r'H2 Model - Input sheet.xlsm')
Inputs = extract_scenario(mapping, 'Scenario1')

##### 1. prepare input data and generate objects
We are testing the following case:

In [3]:
turbine_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Turbine')
display(turbine_data)

issue detected


{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Turbine',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 1495000,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': [],
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.03,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.02,
 'residual_value': 0.01}

In [4]:
Turbine = Object(**turbine_data)
# hier heb je alleen het object met inputs

In [5]:
Turbine.generate_cashflows(startyear=2023, lifecycle=30, debug=True)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

'Escalation years: [2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053]'

'Escalation values: [1.02, 1.0404, 1.061208, 1.08243216, 1.1040808032, 1.126162419264, 1.14868566764928, 1.1716593810022657, 1.195092568622311, 1.2189944199947573, 1.2433743083946525, 1.2682417945625455, 1.2936066304537963, 1.3194787630628724, 1.3458683383241299, 1.3727857050906125, 1.4002414191924248, 1.4282462475762734, 1.4568111725277988, 1.485947395978355, 1.515666343897922, 1.5459796707758806, 1.5768992641913981, 1.6084372494752261, 1.6406059944647307, 1.6734181143540252, 1.7068864766411058, 1.741024206173928, 1.7758446902974065, 1.8113615841033548, 1.847588815785422]'

'CAPEX years: [2023, 2024, 2025]'

'CAPEX values: [-897000000.0, -1794000000.0, -1794000000.0]'

'summed_escalated_capex: -4685224752.0'

...

'CAPEX years escalated: [2023, 2024, 2025, 2053]'

'CAPEX values escalated: [-914940000.0, -1866477600.0, -1903807151.9999998, -173127377.02472454]'

'OPEX value: -163982866.32'

'OPEX years escalated: [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053]'

'OPEX values escalated: [-177500328.19374883, -181050334.75762382, -184671341.4527763, -188364768.28183183, -192132063.64746848, -195974704.92041785, -199894199.01882622, -203892082.99920273, -207969924.65918678, -212129323.1523705, -216371909.61541796, -220699347.80772632, -225113334.76388085, -229615601.45915848, -234207913.48834166, -238892071.75810847, -243669913.19327065, -248543311.4571361, -253514177.68627882, -258584461.2400044, -263756150.4648045, -269031273.4741006, -274411898.9435826, -279900136.92245424, -285498139.66090333, -291208102.4541214, -297032264.50320387, -302972909.7932679]'

#####  2. Test value
##### 2.1 - CAPEX values

In [6]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914940000, -1866477600, -1903807151]}

In [9]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Value in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

Value in 2023: MTBS: -914940000 TUD/VO: -914940000.0
Value in 2024: MTBS: -1866477600 TUD/VO: -1866477600.0
Value in 2025: MTBS: -1903807151 TUD/VO: -1903807151.9999998


##### 2.2 - OPEX values

In [14]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053],
               'values': [-177500328.19374883, -181050334.75762382, -184671341.4527763, -188364768.28183183, -192132063.64746848, -195974704.92041785, -199894199.01882622, -203892082.99920273, -207969924.65918678, -212129323.1523705, -216371909.61541796, -220699347.80772632, -225113334.76388085, -229615601.45915848, -234207913.48834166, -238892071.75810847, -243669913.19327065, -248543311.4571361, -253514177.68627882, -258584461.2400044, -263756150.4648045, -269031273.4741006, -274411898.9435826, -279900136.92245424, -285498139.66090333, -291208102.4541214, -297032264.50320387, -302972909.7932679]}

In [15]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Value in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].opex.item(), 0)

Value in 2026: MTBS: -177500328.19374883 TUD/VO: -177500328.19374883
Value in 2027: MTBS: -181050334.75762382 TUD/VO: -181050334.75762382
Value in 2028: MTBS: -184671341.4527763 TUD/VO: -184671341.4527763
Value in 2029: MTBS: -188364768.28183183 TUD/VO: -188364768.28183183
Value in 2030: MTBS: -192132063.64746848 TUD/VO: -192132063.64746848
Value in 2031: MTBS: -195974704.92041785 TUD/VO: -195974704.92041785
Value in 2032: MTBS: -199894199.01882622 TUD/VO: -199894199.01882622
Value in 2033: MTBS: -203892082.99920273 TUD/VO: -203892082.99920273
Value in 2034: MTBS: -207969924.65918678 TUD/VO: -207969924.65918678
Value in 2035: MTBS: -212129323.1523705 TUD/VO: -212129323.1523705
Value in 2036: MTBS: -216371909.61541796 TUD/VO: -216371909.61541796
Value in 2037: MTBS: -220699347.80772632 TUD/VO: -220699347.80772632
Value in 2038: MTBS: -225113334.76388085 TUD/VO: -225113334.76388085
Value in 2039: MTBS: -229615601.45915848 TUD/VO: -229615601.45915848
Value in 2040: MTBS: -234207913.488341

##### 2.2 - Residual value